In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
import openpyxl  # Usa openpyxl para leitura do Excel

def carregar_arquivo():
    caminho = filedialog.askopenfilename(title="Selecione o arquivo de atendimentos", filetypes=[("Arquivos Excel", "*.xlsx")])
    if caminho:
        try:
            df = pd.read_excel(caminho, engine="openpyxl")  
            return df
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao carregar a planilha: {e}")
            return None
    return None

def processar_tratados():
    df = carregar_arquivo()
    if df is None:
        return
    
    colunas_interesse = ["Usuário Conclusão", "Motivos Conclusão/Retorno Analista", "Mantido Fornecimento"]
    
    for coluna in colunas_interesse:
        if coluna not in df.columns:
            messagebox.showerror("Erro", f"A coluna '{coluna}' não está presente na planilha.")
            return

    df_filtrado = df[colunas_interesse]
    
    motivos_desejados = [
        "Acordo de mão de obra", 
        "Alteração de loja", 
        "Argumentação", 
        "Encomenda", 
        "Item sem giro", 
        "Item fora da cesta"
    ]
    
    df_filtrado = df_filtrado[df_filtrado["Motivos Conclusão/Retorno Analista"].isin(motivos_desejados)]
    
    detalhes_focais = df_filtrado.groupby("Usuário Conclusão")["Mantido Fornecimento"].value_counts().unstack(fill_value=0)
    
    detalhes_focais["Total Atendimentos"] = detalhes_focais.get("SIM", 0) + detalhes_focais.get("NAO", 0)
    
    detalhes_focais["Retenção (%)"] = (detalhes_focais.get("SIM", 0) / detalhes_focais["Total Atendimentos"]) * 100
    detalhes_focais["Retenção (%)"] = detalhes_focais["Retenção (%)"].fillna(0).round(2)

    # Criando os blocos
    bloco_a = detalhes_focais.loc["LUISA BASTOS DE MOURA"]
    bloco_b = detalhes_focais.loc[["YGOR PENA NEGRELLI", "FELIPE JARDIM NOVAES"]].sum()

    equipe = bloco_a + bloco_b

    # Criando um DataFrame com a estrutura correta
    relatorio = pd.DataFrame({
        "Usuário Conclusão": ["EQUIPE", "BLOCO A", "BLOCO B"] + list(detalhes_focais.index),
        "Total de Atendimentos": [equipe["Total Atendimentos"], bloco_a["Total Atendimentos"], bloco_b["Total Atendimentos"]] + list(detalhes_focais["Total Atendimentos"]),
        "Retenção (%)": [equipe["Retenção (%)"], bloco_a["Retenção (%)"], bloco_b["Retenção (%)"]] + list(detalhes_focais["Retenção (%)"])
    })

    # Ordenar por total de atendimentos em ordem decrescente
    relatorio = relatorio.sort_values(by="Total de Atendimentos", ascending=False)

    atualizar_resultado(relatorio)

def atualizar_resultado(relatorio):
    global treeview  
    
    for row in treeview.get_children():
        treeview.delete(row)
    
    for _, row in relatorio.iterrows():
        usuario = row["Usuário Conclusão"]
        total_atendimentos = f"{row['Total de Atendimentos']:.0f}"  
        retencao_percentual = f"{row['Retenção (%)']}%"
        treeview.insert("", "end", values=(usuario, total_atendimentos, retencao_percentual))

# Interface gráfica
root = tk.Tk()
root.title("Automação Acompanhamento Aviãozinho")

btn_processar_tratados = tk.Button(root, text="Processar Dados de Tratados", command=processar_tratados)
btn_processar_tratados.pack(pady=10)

columns = ("Usuário Conclusão", "Total de Atendimentos", "Retenção (%)")
treeview = ttk.Treeview(root, columns=columns, show="headings")

for col in columns:
    treeview.heading(col, text=col)
    treeview.column(col, anchor="center")

treeview.pack(pady=10, padx=20)

root.mainloop()